# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [72]:
# Write your code below.
%load_ext dotenv
%dotenv 

import os
import sys

sys.path.append(os.getenv('SRC_DIR'))
from logger import get_logger
_logs = get_logger (__name__)



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [73]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [98]:
import os
from glob import glob

ft_dir = os.getenv("PRICE_DATA")
ft_dir
ft_glob = glob(os.path.join(ft_dir, '**/*.parquet'), recursive = True)
ft_glob = [f for f in ft_glob if os.path.isfile(f)]
ft_glob
df = dd.read_parquet(ft_glob)
print(df.head())


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.463036   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.142933   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.652882   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.219193   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.237461   

         Volume       sector                       subsector  year  
ticker                                                              
A       4674353  Health Care  Life Sciences Tools & Services  2000  
A       4765083  Health Care  Life Sciences Tools & Services  2000  
A       5758642  Health Care  Life Sciences Tools & Services  2000  
A       2534434  Health Care  Life Sciences Tools & Services  2000  
A       2819626  Health Care  Life Sciences Tools & S

In [99]:
len(df)

2779193

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [116]:

df['Date'] = dd.to_datetime(df['Date'])

df['Close_lag'] = df['Close'].shift(1)
df['Adj Close_lag'] = df['Adj Close'].shift(1)
df['returns'] = (df['Adj Close'] / df['Adj Close_lag']) - 1
df['hi_lo_range'] = df['High'] - df['Low']

dd_feat = df
dd_feat


print(dd_feat.head())
dd_feat

        Date       Open       High        Low      Close  Adj Close   Volume  \
0 2000-01-03  56.330471  56.464592  48.193848  51.502148  43.463036  4674353   
1 2000-01-04  48.730328  49.266811  46.316166  47.567955  40.142933  4765083   
2 2000-01-05  47.389126  47.567955  43.141991  44.617310  37.652882  5758642   
3 2000-01-06  44.080830  44.349072  41.577251  42.918453  36.219193  2534434   
4 2000-01-07  42.247852  47.165592  42.203148  46.494991  39.237461  2819626   

        sector                       subsector  year  Close_lag  \
0  Health Care  Life Sciences Tools & Services  2000        NaN   
1  Health Care  Life Sciences Tools & Services  2000  51.502148   
2  Health Care  Life Sciences Tools & Services  2000  47.567955   
3  Health Care  Life Sciences Tools & Services  2000  44.617310   
4  Health Care  Life Sciences Tools & Services  2000  42.918453   

   Adj_Close_lag   returns  hi_lo_range  Adj Close_lag  
0            NaN       NaN     8.270744            NaN  
1 

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,Adj Close_lag
npartitions=11207,,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [110]:
# Write your code below.
dd_feat_pandas = dd_feat.compute()
dd_feat_pandas['rolling_avg_return'] = dd_feat_pandas['returns'].rolling(window=10).mean()

In [111]:
dd_feat_pandas

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,Adj Close_lag,rolling_avg_return
0,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463036,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,NaN,8.270744,NaN,NaN
1,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142933,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,43.463036,-0.076389,2.950645,43.463036,NaN
2,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652882,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,40.142933,-0.062030,4.425964,40.142933,NaN
3,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219193,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,37.652882,-0.038076,2.771820,37.652882,NaN
4,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237461,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,36.219193,0.083333,4.962444,36.219193,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2024-10-21,192.479996,193.000000,189.179993,189.449997,189.002136,959500,Health Care,Pharmaceuticals,2024,193.279999,192.823074,-0.019816,3.820007,192.823074,0.001221
203,2024-10-22,188.410004,189.820007,187.220001,189.509995,189.061981,1441900,Health Care,Pharmaceuticals,2024,189.449997,189.002136,0.000317,2.600006,189.002136,0.001204
204,2024-10-23,189.399994,189.979996,187.559998,188.990005,188.543228,1339500,Health Care,Pharmaceuticals,2024,189.509995,189.061981,-0.002744,2.419998,189.061981,-0.000564
205,2024-10-24,187.559998,188.250000,180.059998,181.500000,181.070923,4485900,Health Care,Pharmaceuticals,2024,188.990005,188.543228,-0.039632,8.190002,188.543228,-0.004585


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to convert to pandas, the calculation could have been calculated directly in Dask. 
In Dask the same calculations might have been done more efficiently since Dask generally can handle large datasets better then pandas.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.